In [1]:
import torch
from torch.autograd import Variable

In [2]:
x_data= Variable(torch.Tensor([[1.0], [2.0], [3.0]]))
y_data= Variable(torch.Tensor([[2.0], [4.0], [6.0]]))

In [3]:
class LinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(1,1)
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred
    

In [4]:
model = LinearRegressionModel()
# construct our loss function and an optimizer. The call to model.parameters() in the SGD constructor will 
#contain the  learnable parameters of thr two nn.Linear modules which are members of the model
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [5]:
# train
num_epochs = 500
for epoch in range(num_epochs):
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)
    if (epoch+1)%100 == 0:
        print('Epoch [{}/{}], Loss {:.4f}'.format(epoch+1, num_epochs, loss))
    # zero gradients, perform a backward pass, and update the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch [100/500], Loss 0.0029
Epoch [200/500], Loss 0.0007
Epoch [300/500], Loss 0.0002
Epoch [400/500], Loss 0.0000
Epoch [500/500], Loss 0.0000


In [6]:
# test
hour_var = Variable(torch.Tensor([[4.0]]))
y_pred = model(hour_var)
print("predict :", 4, y_pred.data[0][0])

('predict :', 4, tensor(7.9966))


In [7]:
y_pred

tensor([[ 7.9966]])

# Logistic regression

linear regression                                     Logistic regression

\hat y = x * w + b                                    \hat y = \sigma(x*w + b) \sigma(z) = \frac{1}{1-e^{-z}}


loss = 1/N \sum_{i=1}^N (y - \hat y)^2                loss = 1/N \sum_{i=1}^N y_i log \hat y + (1 - y) log (1 -                                                                               \hat y)

In [2]:
from torch import nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable

In [3]:
x_data = Variable(torch.Tensor([[1.0], [2.0], [3.0], [4.0]]))
y_data = Variable(torch.Tensor([[0.0], [0.], [1.], [1.]]))

In [4]:
class LogisticRegressionModel(nn.Module):
    def __init__(self):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(1,1)
    
    def forward(self, x):
        y_pred = F.sigmoid(self.linear(x))
        return y_pred

In [6]:
model = LogisticRegressionModel()
criterion = nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [7]:
num_epochs = 500
for epoch in range(num_epochs):
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)
    if (epoch+1)%100 == 0:
        print('Epoch [{}/{}], Loss {:.4f}'.format(epoch+1, num_epochs, loss))
    # zero gradients, perform a backward pass, and update the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch [100/500], Loss 0.6560
Epoch [200/500], Loss 0.6239
Epoch [300/500], Loss 0.5993
Epoch [400/500], Loss 0.5765
Epoch [500/500], Loss 0.5552


In [8]:
hour_var = Variable(torch.Tensor([[1.0]]))
print("predict 1 hour ", 1.0, model(hour_var).data[0][0] > 0.5)

('predict 1 hour ', 1.0, tensor(0, dtype=torch.uint8))


# Classifying Diabetes

In [10]:
import torch
from torch.autograd import Variable
from torch import nn
import numpy as np

In [11]:
xy = np.loadtxt('../dataset/diabetes.csv.gz', delimiter=',', dtype=np.float32)
x_data = Variable(torch.from_numpy(xy[:,0:-1]))
y_data = Variable(torch.from_numpy(xy[:,[-1]]))
print(x_data.data.shape)
print(y_data.data.shape)

torch.Size([759, 8])
torch.Size([759, 1])


In [12]:
print(x_data.data[0:5,:])
print(y_data.data[0:5])

tensor([[-0.2941,  0.4874,  0.1803, -0.2929,  0.0000,  0.0015, -0.5312,
         -0.0333],
        [-0.8824, -0.1457,  0.0820, -0.4141,  0.0000, -0.2072, -0.7669,
         -0.6667],
        [-0.0588,  0.8392,  0.0492,  0.0000,  0.0000, -0.3055, -0.4927,
         -0.6333],
        [-0.8824, -0.1055,  0.0820, -0.5354, -0.7778, -0.1624, -0.9240,
          0.0000],
        [ 0.0000,  0.3769, -0.3443, -0.2929, -0.6028,  0.2846,  0.8873,
         -0.6000]])
tensor([[ 0.],
        [ 1.],
        [ 0.],
        [ 1.],
        [ 0.]])


In [13]:
class DiabetesModel(nn.Module):
    def __init__(self):
        super(DiabetesModel, self).__init__()
        self.l1 = nn.Linear(8,6)
        self.l2 = nn.Linear(6,4)
        self.l3 = nn.Linear(4,1)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self,x):
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred
    

In [16]:
model = DiabetesModel()
criterion = nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
num_epochs = 500
for epoch in range(num_epochs):
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)
    if (epoch+1)%100 == 0:
        print('Epoch[{}/{}], Loss{:.4f}'.format(epoch+1,num_epochs, loss))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Epoch[100/500], Loss0.8555
Epoch[200/500], Loss0.8335
Epoch[300/500], Loss0.8137
Epoch[400/500], Loss0.7960
Epoch[500/500], Loss0.7802
